In [ ]:
# gptcounter Colab デモ
GPU (A100) を利用して動画を1分解析し、CSV/動画/サマリを出力します。


In [ ]:
# Google Drive をマウントして動画を参照
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)
# 例: ドライブ直下に動画を置いた場合
VIDEO = '/content/drive/MyDrive/merged_20250816_1141-1951.mkv'
os.makedirs('outputs', exist_ok=True)
print('VIDEO=', VIDEO)


In [ ]:
!nvidia-smi || true
!bash scripts/setup_colab.sh


In [ ]:
# A100向け 高速・耐切断プリセットで実行（CSV逐次書き出し）
OUT   = 'outputs/analysis_colab.csv'
!python scripts/analyze_video_mac.py --video "$VIDEO" --start-sec 1800 --duration-sec 3600 \
  --output-csv "$OUT" --video-out outputs/analysis_colab.mp4 --save-video --no-show \
  --reid-cos 0.6 --gate-iou 0.25 --gate-sim 0.45 --det-size 896x896 \
  --body-conf 0.5 --w-face 0.7 --w-body 0.3 --detect-every-n 2 --device cuda \
  --tracker bytetrack \
  --log-every-sec 5 --checkpoint-every-sec 15 --flush-every-n 10 --no-merge --merge-every-sec 0 \
  --run-id colab_fast | cat


In [ ]:
VIDEO = '/content/merged_20250816_1141-1951.mkv'  # 例: ドライブに置く
OUT   = 'outputs/analysis_colab.csv'
!python scripts/analyze_video_mac.py --video "$VIDEO" --start-sec 1800 --duration-sec 60 \
  --output-csv "$OUT" --video-out outputs/analysis_colab.mp4 --save-video --no-show \
  --reid-cos 0.6 --gate-iou 0.25 --gate-sim 0.45 --det-size 1024x1024 \
  --body-conf 0.5 --w-face 0.7 --w-body 0.3 --detect-every-n 1 --device cuda \
  --log-every-sec 5 --checkpoint-every-sec 60 --merge-every-sec 60 --run-id colab | cat


In [ ]:
!python scripts/person_summary.py --input-csv outputs/analysis_colab.csv \
  --video "$VIDEO" --output-csv outputs/person_summary_colab.csv \
  --cluster-from person --emb-thresh 0.55 --overlap-veto-sec 0.6 \
  --merge-max-dist-px 80 --stats-out outputs/person_stats_colab.csv | cat
import pandas as pd
summary=pd.read_csv('outputs/person_summary_colab.csv')
print('unique=', len(summary))
summary.head()
